<a href="https://colab.research.google.com/github/fafnirkyu/data-challenge-alura/blob/main/semana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 63.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2fb83190af615739fbbfce3447074064dd71f3911086d4aca34d30b253deee9f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


# importação das bibliotecas a serem usadas

In [ ]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.functions import vector_to_array
import plotly.express as px
from scipy.spatial.distance import euclidean
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType
from pyspark.ml import Pipeline
import numpy as np

In [ ]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [ ]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip" && unzip semanas-3-e-4.zip -d dados/

--2022-12-09 14:35:48--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.142.41, 52.217.136.57, 52.216.107.124, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.142.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip’

semanas-3-e-4.zip   100%[===================>]   2.47M  --.-KB/s    in 0.1s    

2022-12-09 14:35:48 (18.6 MB/s) - ‘semanas-3-e-4.zip’ saved [2588308/2588308]

Archive:  semanas-3-e-4.zip
   creating: dados/dataset_ml_parquet/
  inflating: dados/dataset_ml_parquet/_SUCCESS  
  inflating: dados/dataset_ml_parquet/._SUCCESS.crc  
  inflating: dados/dataset_ml_parquet/part-00003-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: dados/dataset_ml_parquet/part-00001-a14b227c-f87e-4893-b5f

In [ ]:
dados = spark.read.parquet('/content/dados/dataset_ml_parquet')

In [ ]:
dados.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [ ]:
X = dados.columns
X.remove('bairro')
X.remove('id')
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'valor',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

# Vetorização dos dados

In [ ]:
dados_vector = VectorAssembler(inputCols=X, outputCol='features').transform(dados)
dados_vector.show(truncate=False, n=5)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|features                                                                                                    |
+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+-------

# Standard Scaler

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dados_vector)
dados_scaler = scaler_model.transform(dados_vector)
dados_scaler.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fech

# PCA

In [ ]:
pca = PCA(k=len(X), inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_scaler)
dados_pca = model_pca.transform(dados_scaler)
dados_pca.select('pca_features').show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------

#K-means

In [ ]:
SEED= 150
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca').setK(3).setSeed(SEED)
model_kmeans = kmeans.fit(dados_pca)
projections_kmeans = model_kmeans.transform(dados_pca)

# Distribuição dos clusters

In [ ]:
pca_features_xy = projections_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                .withColumn('y', vector_to_array('pca_features')[1])\
                .select(['x', 'y', 'cluster_pca', 'valor'])
fig = px.scatter(pca_features_xy.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['x', 'y', 'valor'])
fig.show()

#Filtro de imóveis pelo cluster

In [ ]:
id = projections_kmeans.select('id').collect()[0][0]
cluster = projections_kmeans.filter(projections_kmeans.id == id).select('cluster_pca').collect()[0][0]
imoveis_recomendados = projections_kmeans.filter(projections_kmeans.cluster_pca == cluster)
imoveis_recomendados.show(10)

+--------------------+-----+---------+---------+-------+------+----+--------------------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+-----------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|              bairro|condominio|  iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|     scaled_features|        pca_features|cluster_pca|
+--------------------+-----+---------+---------+-------+------+----+--------------------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+----

In [ ]:
componentes = imoveis_recomendados.filter(imoveis_recomendados.id == id).select('pca_features').collect()[0][0]
componentes   

DenseVector([-6.1651, 1.3381, -1.7052, -0.5338, 0.089, -0.3134, -5.8807, -4.3463, -0.1423, -0.3627, 0.0853, 1.1262, -0.7941, -1.072, -0.0575, -0.2278, -0.1392, 0.0299, -0.4594, -0.2334, 0.0908, 0.2367, -1.2629])

#Cálculo de distância euclideana

In [ ]:
def calcula_distance(value):
  return euclidean(componentes, value)

udf_calcula_distance = f.udf(calcula_distance, FloatType())

musicas_recomendadas_dist = imoveis_recomendados.withColumn('Dist', udf_calcula_distance('pca_features'))

recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['id', 'bairro', 'Dist'])

recomendadas.show()

+--------------------+------------+-------------------+
|                  id|      bairro|               Dist|
+--------------------+------------+-------------------+
|00002dd9-cc74-480...|Santo Cristo|                0.0|
|5dac977a-6cfb-428...|      Centro|0.03989782929420471|
|33aeb3e6-f85d-49b...|      Centro| 0.0631047785282135|
|a1c430e5-fe77-4b0...|      Centro|0.06587200611829758|
|b3d1d649-f026-48a...|      Centro|0.08980962634086609|
|a99baaa1-00ef-490...|      Centro|0.09723300486803055|
|842a87a0-7602-4b8...|      Centro|0.11216022819280624|
|d4a64276-c147-4b3...|      Centro|0.11537354439496994|
|562af84b-6b6d-422...|Santo Cristo|  0.136283278465271|
|1238503b-dbbc-466...|Santo Cristo| 0.1365862488746643|
+--------------------+------------+-------------------+



#Função recomendadora

In [ ]:
def recomendador(nome_musica):
  cluster = projections_kmeans.filter(projections_kmeans.id == id).select('cluster_pca').collect()[0][0]
  imoveis_recomendados = projections_kmeans.filter(projections_kmeans.cluster_pca == cluster)
  componentes = imoveis_recomendados.filter(imoveis_recomendados.id == id).select('pca_features').collect()[0][0]
  def calcula_distance(value):
    return euclidean(componentes, value)
  udf_calcula_distance = f.udf(calcula_distance, FloatType())
  imoveis_recomendados = imoveis_recomendados.withColumn('Dist', udf_calcula_distance('pca_features'))
  recomendadas = spark.createDataFrame(imoveis_recomendados.sort('Dist').take(10)).select(dados.columns)
  recomendadas.select(dados.columns).show(truncate=False)

In [ ]:
recomendador('d4a64276-c147-4b39-b5ce-19d03fbdb744')

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-4809-b5a5-850adf0e7526|2    |35       |1        |1      |0.0   |0.0 |Sant

#Criação do pipeline

In [ ]:
pipeline = Pipeline(stages =[VectorAssembler(inputCols=X, outputCol='features'),
                              StandardScaler(inputCol='features', outputCol='scaled_features'),
                              PCA(k=len(X), inputCol='scaled_features', outputCol='pca_features'),
                              KMeans(featuresCol='pca_features', predictionCol='cluster_pca').setK(6).setSeed(SEED)])
pipeline_kmeans_model = pipeline.fit(dados)
dados_kmeans = pipeline_kmeans_model.transform(dados)
dados_kmeans.show(5, truncate = True)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+-----------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|     scaled_features|        pca_features|cluster_pca|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---

# Cálculo do número ideal de componentes

In [ ]:
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(len(X))]
lista_valores
k = sum(np.array(lista_valores) <= 0.7)
print('O número ideal de componentes é:', k)

O número ideal de componentes é: 6
